In [49]:
# Import Libraries
import numpy as np
import pandas as pd
import tensorflow as tf

In [50]:
# Constants
# Data Location
LOC = "D:\DSCI\Limited_set"
Model_LOC = ""
# Classes to use
classes = {

}

In [ ]:
# Model Dictionary
models = {
    "Xception": tf.keras.applications.xception.Xception(weights="imagenet"),
    "VGG16": tf.keras.applications.vgg16.VGG16(weights="imagenet"),
    "VGG19": tf.keras.applications.vgg19.VGG19(weights="imagenet"),
    "InceptionV3": tf.keras.applications.InceptionV3(weights="imagenet"),
    "ResNet50": tf.keras.applications.resnet50.ResNet50(weights="imagenet"),
#    "AlexNet": tf.keras.models.load_model("")
}

# PreProcessing System
preproc = {
    "Xception": tf.keras.applications.xception.preprocess_input,
    "VGG16": tf.keras.applications.vgg16.preprocess_input,
    "VGG19": tf.keras.applications.vgg19.preprocess_input,
    "InceptionV3": tf.keras.applications.inception_v3.preprocess_input,
    "ResNet50": tf.keras.applications.resnet50.preprocess_input,
#    "AlexNet": tf.keras.models.load_model("")
}


In [ ]:
# Testing
def test_model(model):
    print("Running tests on model:", model)
    count = 0
    for i in train:
        for j in range(len(i[1])):
            a = i[0][j]
            a = np.expand_dims(a, axis=0)
            b = np.copy(a)
            c = preproc[model](b)
            res = models[model].predict(c)
            label = tf.keras.applications.imagenet_utils.decode_predictions(res)
            # print(int(i[1][j]))
            print(label)
            print(train.class_names[int(i[1][j])])
            print("\n\n")
        break
    # for j in test:
    #     res = models[model].predict(j[0])


In [ ]:
for model1 in models:
    shapes = models[model1].get_config()["layers"][0]["config"]["batch_input_shape"]
    shape = (shapes[1], shapes[2])
    train, test = tf.keras.preprocessing.image_dataset_from_directory(LOC, validation_split=.2, subset="both", seed=42, image_size=shape, batch_size=32)
    # for element in train:
    #     print(element[1])
    #     break

    try:
        test_model(model1)

    except Exception as e:
        print(e)
    break